In [3]:
import sys; print(sys.version)

3.11.0 (main, Oct 24 2022, 18:26:48) [MSC v.1933 64 bit (AMD64)]


In [4]:
!pip install --upgrade pip

     ---------------------------------------- 1.8/1.8 MB 7.8 MB/s eta 0:00:00


ERROR: To modify pip, please run the following command:
C:\Users\avane\Desktop\brain_tumor_detection_CNN\myenv-311\Scripts\python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install tensorflow

     -------------------------------------- 375.9/375.9 MB 7.1 MB/s eta 0:00:00
     -------------------------------------- 135.6/135.6 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB ? eta 0:00:00
     --------------------------------------- 26.4/26.4 MB 16.8 MB/s eta 0:00:00
     ---------------------------------------- 71.9/71.9 kB 3.9 MB/s eta 0:00:00
     ------------------------------------- 434.5/434.5 kB 26.5 MB/s eta 0:00:00
     ---------------------------------------- 4.3/4.3 MB 12.4 MB/s eta 0:00:00
     ---------------------------------------- 5.5/5.5 MB 16.0 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 17.1 MB/s eta 0:00:00
     --------------------------------------- 12.9/12.9 MB 19.2 MB/s eta 0:00:00
     ---------------------------------------- 3.0/3.0 MB 21.1 MB/s eta 0:00:00
     ------------------------------------- 209.7/209.7 kB 12.5 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1


[notice] A new release of pip available: 22.3 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from IPython.display import clear_output
!pip install kaggle imutils plotly opencv-python matplotlib scikit-learn
clear_output()

In [7]:
import numpy as np
from tqdm import tqdm
import cv2
import os
import shutil
import itertools
import imutils
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import tensorflow as tf
from tensorflow import keras

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools

# from keras.preprocessing.image import ImageDataGenerator
# from keras.applications.vgg16 import VGG16, preprocess_input
# from keras import layers
# from keras.models import Model, Sequential
# from keras.optimizers import Adam, RMSprop
# from keras.callbacks import EarlyStopping

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping


init_notebook_mode(connected=True)
RANDOM_SEED = 123

In [8]:
import zipfile

# Download datasets
!kaggle datasets download -d gaborfodor/keras-pretrained-models
!kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection

# Unzip the datasets
with zipfile.ZipFile("keras-pretrained-models.zip", 'r') as zip_ref:
    zip_ref.extractall("keras_pretrained_models")

with zipfile.ZipFile("brain-mri-images-for-brain-tumor-detection.zip", 'r') as zip_ref:
    zip_ref.extractall("brain_mri_images")

print("Data source import complete.")


Dataset URL: https://www.kaggle.com/datasets/gaborfodor/keras-pretrained-models
License(s): CC-BY-SA-4.0
keras-pretrained-models.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset URL: https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection
License(s): copyright-authors
brain-mri-images-for-brain-tumor-detection.zip: Skipping, found more recently modified local copy (use --force to force download)
Data source import complete.


In [9]:
import os

# Create folders
folders = [
    'TRAIN/YES', 'TRAIN/NO',
    'TEST/YES', 'TEST/NO',
    'VAL/YES', 'VAL/NO'
]

for folder in folders:
    os.makedirs(folder, exist_ok=True)

# Display folder tree
print("\nFolder structure:")
for root, dirs, files in os.walk(".", topdown=True):
    for name in dirs:
        print(os.path.join(root, name))



Folder structure:
.\.git
.\.ipynb_checkpoints
.\brain_mri_images
.\brain_tumor_dataset
.\keras_pretrained_models
.\myenv-311
.\no
.\TEST
.\TRAIN
.\VAL
.\venv
.\yes
.\.git\hooks
.\.git\info
.\.git\logs
.\.git\objects
.\.git\refs
.\.git\logs\refs
.\.git\logs\refs\heads
.\.git\logs\refs\remotes
.\.git\logs\refs\remotes\origin
.\.git\objects\23
.\.git\objects\45
.\.git\objects\47
.\.git\objects\59
.\.git\objects\6a
.\.git\objects\6c
.\.git\objects\9e
.\.git\objects\aa
.\.git\objects\d4
.\.git\objects\e0
.\.git\objects\info
.\.git\objects\pack
.\.git\refs\heads
.\.git\refs\remotes
.\.git\refs\tags
.\.git\refs\remotes\origin
.\brain_mri_images\brain_tumor_dataset
.\brain_mri_images\no
.\brain_mri_images\yes
.\brain_mri_images\brain_tumor_dataset\no
.\brain_mri_images\brain_tumor_dataset\yes
.\brain_tumor_dataset\no
.\brain_tumor_dataset\yes
.\myenv-311\etc
.\myenv-311\Include
.\myenv-311\Lib
.\myenv-311\Scripts
.\myenv-311\share
.\myenv-311\etc\jupyter
.\myenv-311\etc\jupyter\jupyter_notebo

In [10]:
import shutil
IMG_PATH = './brain_tumor_dataset/'
# split the data by train/val/test
for CLASS in os.listdir(IMG_PATH):
    if not CLASS.startswith('.'):
        IMG_NUM = len(os.listdir(IMG_PATH + CLASS))
        for (n, FILE_NAME) in enumerate(os.listdir(IMG_PATH + CLASS)):
            img = IMG_PATH + CLASS + '/' + FILE_NAME
            if n < 5:
                shutil.copy(img, 'TEST/' + CLASS.upper() + '/' + FILE_NAME)
            elif n < 0.8*IMG_NUM:
                shutil.copy(img, 'TRAIN/'+ CLASS.upper() + '/' + FILE_NAME)
            else:
                shutil.copy(img, 'VAL/'+ CLASS.upper() + '/' + FILE_NAME)

In [11]:
def load_data(dir_path, img_size=(100,100)):
    """
    Load resized images as np.arrays to workspace
    """
    X = []
    y = []
    i = 0
    labels = dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith('.'):
                    img = cv2.imread(dir_path + path + '/' + file)
                    X.append(img)
                    y.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print(f'{len(X)} images loaded from {dir_path} directory.')
    return X, y, labels



def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize = (6,6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    cm = np.round(cm,2)
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()